In [34]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [35]:
def compute_CUSUM(X):
    CUSUM = np.cumsum(X ** 2)
    return CUSUM


def compute_gamma(X, T, m):
    mean_X = np.mean(X)
    r = X - mean_X  # 잔차 계산
    r_squared = r ** 2
    sigma_squared = np.mean(r_squared)

    gamma = np.zeros(m + 1)
    for i in range(0, m + 1):
        gamma_i = np.sum((r_squared[i:T] - sigma_squared) * (r_squared[0:T - i] - sigma_squared))
        gamma[i] = gamma_i / T
    return gamma


def compute_lambda(gamma, m):
    lambda_hat = gamma[0] + 2 * np.sum((1 - np.arange(1, m + 1) / (m + 1)) * gamma[1:m + 1])
    return lambda_hat


def compute_D_prime(CUSUM, T, lambda_hat):
    k = np.arange(T)
    D_prime = (CUSUM - (k + 1) / T * CUSUM[-1]) / np.sqrt(lambda_hat)
    return D_prime


def compute_percentile(D_prime, T, percent=95):
    D_prime_abs = np.abs(D_prime) * np.sqrt(T / 2)
    critical_value = np.percentile(D_prime_abs, percent)
    return critical_value

In [36]:
def set_plot_params(title, ylabel):
    plt.title(title)
    plt.ylabel(ylabel, fontsize=12)
    plt.xlabel("Date", fontsize=12)
    plt.yticks(fontsize=12)
    plt.xticks(rotation=30, fontsize=12)
    plt.gca().xaxis.set_major_locator(plt.MultipleLocator(30))


def calculate_ylim(data):
    ylim_min = min(data) - (max(data) - min(data)) * 0.1
    ylim_max = max(data) + (max(data) - min(data)) * 0.1
    return ylim_min, ylim_max


def draw_graph(data: pd.DataFrame, title: str, ylabel: str, date_lst: list, color="blue"):
    '''
    :param data: 시간 인덱스와 그래프로 표시할 정보가 있는 데이터프레임
    :param title: 그래프 제목
    :param ylabel: 그래프 y축 단위
    :param date_lst: 아웃라이어 날짜인덱스 리스트
    :param color: 그래프 색깔
    :return: 
    '''
    plt.figure(figsize=(10, 5))
    plt.plot(data.index, data.values, color=color)
    for date in date_lst:
        plt.axvline(pd.to_datetime(date), color='red', linestyle='--', label='Highlight Date')
    set_plot_params(title, ylabel)
    ylim_min, ylim_max = calculate_ylim(list(data.values))
    plt.ylim(ylim_min, ylim_max)
    plt.show()

In [37]:
def ol_detect(hour_df, diff_value, window_size, significant_level, number):
    lambda_lst = []
    cv_lst = []
    current_lst = []
    ol_lst = []

    for i in tqdm(range(window_size, len(diff_value) + 1, 1)):

        count = 0
        filter = []
        for val in ol_lst:
            if (i - window_size <= val) and (val < i):
                count += 1
                filter.append(val - (i))

        X = diff_value[i - window_size - count: i].copy()

        if len(filter) != 0:
            X = np.delete(X, filter)

        N = len(X)
        T = N
        m = int(T ** (1 / 4))

        # CUSUM 계산
        CUSUM = compute_CUSUM(X)
        # gamma 계산
        gamma = compute_gamma(X, T, m)
        # lambda_hat 계산
        lambda_hat = compute_lambda(gamma, m)
        lambda_lst.append(lambda_hat)
        # D_prime 계산
        D_prime = compute_D_prime(CUSUM, T, lambda_hat)
        # critical value 계산
        critical_value = compute_percentile(D_prime, T, significant_level)
        cv_lst.append(critical_value)
        # 현재 통계량 계산
        current_lst.append(np.abs(D_prime[-2]) * np.sqrt(T / 2))

        if current_lst[-1] > critical_value:
            ol_lst.append(i - 1)

    ol_lst = list(np.where((np.array(current_lst) > np.array(cv_lst)))[0] + window_size)
    print(len(ol_lst))

    plt.figure(figsize=(13, 2), dpi=400)
    plt.plot(hour_df[name], color='tab:blue')

    for point in ol_lst:
        plt.axvline(hour_df.index[point], color='tab:red', linewidth=0.5, linestyle='--')
        plt.scatter(hour_df.index[point], hour_df[name].iloc[point], color='tab:red', s=10)

    plt.xlabel('Date')
    plt.ylabel('KM')  # y축 레이블과 단위
    # plt.savefig(f'File/images/{number}/{window}_{level}.png')
    plt.show()
    plt.close()

In [38]:
import os

input_dir = 'Database'
output_dir = 'File/images'

number_lst = [39227, 44343, 44349, 44350, 44351, 44353, 44358, 58464, 46267, 53611, 53019, 51961, 47775, 51969, 58722,
              56361, 56289, 56783, 48018, 43823, 45246, 29349, 37265, 42691, 42984, 55841]

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# List of files that do not exist in the output directory
missing_dirs = [str(file) for file in number_lst if not os.path.exists(f"{output_dir}/{file}")]

# Create missing files
for directory in missing_dirs:
    dir_path = os.path.join(output_dir, directory)
    os.makedirs(dir_path)

print(f"Created {len(missing_dirs)} missing files in {output_dir}.")

Created 0 missing files in File/images.


In [39]:
LEO_lst = [39227, 44343, 44349, 44350, 44351, 44353, 44358, 58464, 46267, 53611, 53019, 51961, 47775, 51969, 58722,
           56361, 56289, 56783, 48018]
GEO_lst = [43823, 45246, 29349, 37265, 42691, 42984, 55841]

# Preprocessing Old

In [ ]:
for number in LEO_lst:
    name = 'perigee'
    info_df = pd.read_csv(f'Database/{number}.csv')
    df_1 = info_df.copy()
    df_1.loc[:, 'time'] = pd.to_datetime(df_1[['year', 'month', 'day', 'hour', 'minute']])
    df_1.set_index('time', inplace=True)
    df_1.drop(columns=['year', 'month', 'day', 'hour', 'minute'], inplace=True)
    hour_df = df_1[::60].copy()
    diff_value = np.log(hour_df[name].copy()).diff().fillna(0).values

    for level in [95, 99]:
        for window in [24 * 7 * 1, 24 * 7 * 2, 24 * 7 * 4]:
            print(f'number: {number}, window : {window}, level : {level}')
            ol_detect(hour_df, diff_value, window, level, number)

In [ ]:
for number in GEO_lst:
    name = 'longitude'
    info_df = pd.read_csv(f'Database/{number}.csv')

    df_1 = info_df.copy()
    df_1.loc[:, 'time'] = pd.to_datetime(df_1[['year', 'month', 'day', 'hour', 'minute']])
    df_1 = df_1[df_1.index > '2022-12-31 23:59:59'].copy()
    df_1.set_index('time', inplace=True)
    df_1.drop(columns=['year', 'month', 'day', 'hour', 'minute'], inplace=True)
    hour_df = df_1[::60].copy()
    diff_value = np.log(hour_df[name].copy()).diff().fillna(0).values

    for level in [95, 99]:
        for window in [24 * 7 * 1, 24 * 7 * 2, 24 * 7 * 4]:
            print(f'number: {number}, window : {window}, level : {level}')
            ol_detect(hour_df, diff_value, window, level, number)

# Preprocessing New

In [0]:
for number in LEO_lst:
    name = 'perigee'
    info_df = pd.read_csv(f'Database/{number}_new.csv', index_col=0)
    info_df = info_df[info_df.index > '2022-12-31 23:59:59'].copy()
    hour_df = info_df[::60].copy()
    diff_value = np.log(hour_df[name].copy()).diff().fillna(0).values

    start_date = hour_df.index[0]
    data_length = len(hour_df)
    time_index = pd.date_range(start=start_date, periods=data_length, freq='h')
    hour_df.index = time_index

    for level in [95, 99]:
        for window in [24 * 7 * 1, 24 * 7 * 2, 24 * 7 * 4]:
            print(f'number: {number}, window : {window}, level : {level}')
            ol_detect(hour_df, diff_value, window, level, number)

In [0]:
for number in GEO_lst:
    name = 'longitude'
    info_df = pd.read_csv(f'Database/{number}_new.csv', index_col=0)
    info_df = info_df[info_df.index > '2022-12-31 23:59:59'].copy()
    hour_df = info_df[::60].copy()
    diff_value = np.log(hour_df[name].copy()).diff().fillna(0).values

    start_date = hour_df.index[0]
    data_length = len(hour_df)
    time_index = pd.date_range(start=start_date, periods=data_length, freq='h')
    hour_df.index = time_index

    for level in [95, 99]:
        for window in [24 * 7 * 1, 24 * 7 * 2, 24 * 7 * 4]:
            print(f'number: {number}, window : {window}, level : {level}')
            ol_detect(hour_df, diff_value, window, level, number)

# 미분값

In [ ]:
fig, axes = plt.subplots(8, 1, figsize=(12, 10), dpi=400, sharex=True)

for i, col_name in enumerate(
        ['altitude', 'velocity', 'apogee', 'perigee', 'inclination', 'eccentricity', 'raan', 'longitude']):
    value = hour_df[col_name].values
    diff_value = np.abs(hour_df[col_name].values)

    axes[i].plot(hour_df.index, diff_value, color='tab:orange')
    #axes[i].set_ylabel(col_name)
    axes[i].legend([col_name], loc='upper left', fontsize=8)

    # for point in ol_lst:
    #     axes[i].axvline(hour_df.index[point], color='tab:red', linewidth=0.5, linestyle='--')
    #     axes[i].scatter(hour_df.index[point], hour_df[col_name].iloc[point], color='tab:red', s=10)

axes[-1].set_xlabel('Date')
plt.tight_layout()
plt.show()